뉴스 Summary(2줄 뉴스) 를 konlpy 로 토크나이즈 해서 DB에 날짜 이름의 테이블로 저장
- team1_project.20230101
- table schema: (summary, press, rdate, tokenized_summary)
- 코랩 실행시 단어 추가하고 Restart session 실행


In [ ]:
# 코랩 실행시 맨 처음 설치 필요
!pip install JPype1
!pip install konlpy
!pip install pyMySQL
!pip install -U finance-datareader

In [1]:
# import packages
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# --konlpy 커스터마이징  ----------------------------------------------
# --------------------------------------------------------------------
# Konlpy에 한국거래소 상장종목 전체 종목명과 약명을 새로운 단어로 추가하기
# - 코랩에서 작업시 매번 실행, GCP 에서는 한번만 실행
# --------------------------------------------------------------------
# konlpy 디렉토리로 이동
os.chdir('konlpy/설치된/디렉토리/java')
# 현재 디렉토리 확인
os.getcwd()
# 압축 풀 임시 디렉토리 /temp 만들기
os.makedirs('./temp')
# 임시디렉토리로 이동
os.chdir('./temp')
# 압축 풀기
!jar xvf ../open-korean-text-2.1.0.jar

# FinanceDataReader로 한국거래소 상장종목 전체 종목을 가져오기
df_krx = fdr.StockListing('KRX')
# 종목명만 추출하기
name_list = df_krx['Name']
# 새로운 단어목록 생성을 위한 data 변수 선언
data = ''
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 새로운 단어로 넣기
# DB 연결 준비: 종목명 약어 읽어오기
conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
# DB에서 종목병 약어 추출위한 sql 준비
sql = f"SELECT x.kor_name,x.kor_abb FROM DB이름.테이블이름 x"
# DB 검색결과를 dataframe에 저장
name_abb_df = pd.read_sql_query(sql, conn)
# 실행확인을 위한 화면 출력
print(name_abb_df.head(3))
# DB close
conn.close()
# 종목명 추가하기
name_list = name_abb_df['kor_name']
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 추가하기
name_list = name_abb_df['kor_abb']
# 종목명 약명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# konlpy의 company_names.txt 사전으로 저장
with open("konlpy/설치된/디렉토리/java/temp/org/openkoreantext/processor/util/noun/company_names.txt", 'w') as f:
    f.write(data)
# 임시 작업 디렉토리로 이동 확인
os.chdir('konlpy/설치된/디렉토리/java/temp')
# 다시 압축
!jar cvf ../open-korean-text-2.1.0.jar *
# 코랩 실행시 Restart session 실행

# --konlpy 커스터마이징 END --------------------------------------------

In [ ]:
# --konlpy 커스터마이징 확인 ----------------------------------------------
# konlpy 커스터마이징 확인용 코드
# 종목명 토크나이즈 확인할 것: 삼성전자 -> '삼성','전자'(X), '삼성전자'(O)
# 제대로 안되면 konlpy 커스터마이징 확인
import konlpy
from konlpy.tag import Okt

okt = Okt()
print(okt.pos("삼성전자 005930 "))
# --konlpy 커스터마이징 확인 End ------------------------------------------

In [3]:
# -- tokenize_oneday_news_summary() -------------------------------------
# 특정일의 뉴스 summary 를 tokenize 하기
# 특정일 날짜: YYYYMMDD ex) 20240105 (문자열)
def tokenize_oneday_news_summary(sdate):
  # DB 연결 준비: 뉴스(2줄 속보) 읽어오기
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 뉴스(summary, press, rdate) 추출위한 sql 준비
  sql = f"SELECT x.summary,x.press, x.rdate FROM DB이름.{sdate} x"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  #print(result_df.head(3))
  # DB close
  conn.close()

  # konlpy의 okt 생성
  okt=Okt()

  # okt로 tokenize해서 형태소 품사 morph 추출하기 (pos 는 okt.pos, 명사는 okt.nouns)
  result_df['morphs_tokenized_summary'] = result_df['summary'].apply(okt.morphs)
  # DB 저장을 위해 문자열로 변환
  result_df['morphs_tokenized_summary'] = result_df['morphs_tokenized_summary'].astype(str)

  # okt로 tokenize해서 형태소 pos 추출하기 (품사 전체는 okt.morphs, 명사는 okt.nouns)
  result_df['pos_tokenized_summary'] = result_df['summary'].apply(okt.pos)
  
  # DB 저장을 위해 문자열로 변환
  result_df['pos_tokenized_summary'] = result_df['pos_tokenized_summary'].astype(str)

  # okt로 tokenize해서 명사 형태소 nouns 추출하기 (품사 전체는 okt.morphs)
  result_df['nouns_tokenized_summary'] = result_df['summary'].apply(okt.nouns)
  
  # DB 저장을 위해 문자열로 변환
  result_df['nouns_tokenized_summary'] = result_df['nouns_tokenized_summary'].astype(str)

  # pos tokenized 에서 Modifier,Josa,Suffix,Punctuation,Foreigh 제외, 1 자리 단어 제외
  result_df['tokenized_summary'] = ""
  for i in range(len(result_df)):
    # 새로운 문자열 위한 리스트
    new_str = []
    # 튜플 형태로 꺼내면 (단어,품사) 형태이므로 뒤세 품사를 보고 맞는 단어들만 새로운 문자열로 만든다
    for (x,y) in ast.literal_eval(result_df['pos_tokenized_summary'][i]):
      # 관형사,조사,접두사,문장부호,한자및 기타기호 제외
      if (y!='Modifier')&(y!='Josa')&(y!='Suffix')&(y!='Punctuation')&(y!='Foreign'):
        # 한글자 짜리 제외
        if(len(str(x))>1):
          new_str.append(x)
    # 실행 확인을 위한 화면 출력
    #print(new_str)
    # 토크나이즈된 문자열 데이터프레임에 넣기
    result_df['tokenized_summary'][i]=str(new_str)

  # DB 저장을 위해서 필요한 컬럼 정리
  result_df = result_df[['summary','press','rdate','tokenized_summary','pos_tokenized_summary']]
  # 실행 확인을 위한 화면 출력
  #print(result_df.head(3))

  # DB 저장을 위한 연결 준비
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  db_connection = create_engine(db_connection_path)
  # DB 커넥션 생성
  conn = db_connection.connect()
  # 데이터프레임을 DB로 저장, 날짜 ddate_sokbo를 테이블 명으로, 이미 있으면 덮어쓰기(replace)
  result_df.to_sql(sdate+'_sokbo',conn,if_exists='replace')
  # DB 커밋
  conn.commit()
  # DB close
  conn.close()
# -- tokenize_oneday_news_summary() :END------------------------------------------------

# -- tokenize_news_summary() -----------------------------------------------------------
# 지정된 기간의 네이버증권실시간속보 뉴스 토크나이즈 하기
# 날짜는 'YYYYMMDD'형식 문자열
# 시작날짜 startDate, 종료날짜 endDate 를 파라미터로 받기
def tokenize_news_summary(startDate,endDate):
  # 시작날짜 ddate 복사
  ddate = startDate
  # 종료 날짜까지 반복하기
  while pd.to_datetime(ddate).date() <= pd.to_datetime(endDate).date():
    # 해당일 하루 뉴스 tokenize
    tokenize_oneday_news_summary(ddate)
    # 실행중 확인을 위한 날짜 화면 출력
    print(ddate)
    # 자동 날짜 증가 연산: 1일씩 증가
    dt_date = pd.to_datetime(ddate) + pd.DateOffset(days=1)
    # 다음 날짜 문자열 추출
    ddate= dt_date.strftime("%Y%m%d")
# -- getNewsFromNaver() :END--------------------------------------------------------


In [ ]:
#===================================================================================
# --메인 함수 호출 ------------------------------------------------------------------
# tokenize할 뉴스의 기간을 파라미터로 넘기기
# 예) 시작날짜 '20240117', 종료날짜 '20240119'인 경우
# tokenize_news_summary('20240117','20240119') 로 함수 호출
#-----------------------------------------------------------------------------------
tokenize_news_summary('20240117','20240119')
#====================================================================================
